## Cognitive Battery Introduction: Jax-3DP3

In [ ]:
import os
import numpy as np
import jax3dp3
from jax3dp3.viz import (
    get_depth_image,
    multi_panel,
)
from jax3dp3.transforms_3d import transform_from_pos, unproject_depth
import jax.numpy as jnp
import jax
from jax3dp3.viz import make_gif_from_pil_images
from PIL import Image
from tqdm import tqdm

import cog_utils as utils

In [ ]:
scene = "swap"
data_path = f"/home/khaledshehada/cog_jax3dp3_data/{scene}_data/videos/"
num_frames = len(os.listdir(os.path.join(data_path, "frames")))

In [ ]:
width = 300
height = 300
fov = 90

fx, fy, cx, cy = utils.get_camera_intrinsics(width, height, fov)
near, far = 0.001, 50.0

intrinsics = jax3dp3.Intrinsics(height, width, fx, fy, cx, cy, near, far)
renderer = jax3dp3.Renderer(intrinsics=intrinsics)

In [ ]:
rgb_images, depth_images, seg_maps = [], [], []
rgb_images_pil = []
for i in range(num_frames):
    rgb_path = os.path.join(data_path, f"frames/frame_{i}.jpeg")
    if not os.path.isfile(rgb_path):
        rgb_path = rgb_path.replace("jpeg", "png")
    rgb_img = Image.open(rgb_path)
    rgb_images_pil.append(rgb_img)
    rgb_images.append(np.array(rgb_img))

    depth_path = os.path.join(data_path, f"depths/frame_{i}.npy")
    depth_npy = np.load(depth_path)
    depth_images.append(depth_npy)

    seg_map = np.load(os.path.join(data_path, f"segmented/frame_{i}.npy"))
    seg_maps.append(seg_map)

In [ ]:
coord_images = []  # depth data in 2d view as images
seg_images = []  # segmentation data as images

for frame_idx in tqdm(range(num_frames)):
    coord_image = np.array(unproject_depth(depth_images[frame_idx], intrinsics))
    segmentation_image = seg_maps[frame_idx].copy()
    mask = np.invert(
        (coord_image[:, :, 0] < 2.0)
        * (coord_image[:, :, 0] > -1)
        * (coord_image[:, :, 1] < 0.463)
        * (coord_image[:, :, 1] > -0.8)
        * (coord_image[:, :, 2] < 1.8)
        * (coord_image[:, :, 2] > 0.25)
    )
    coord_image[mask, :] = 0.0
    segmentation_image[mask, :] = 0.0
    coord_images.append(coord_image)
    seg_images.append(segmentation_image)

coord_images = np.stack(coord_images)
seg_images = np.stack(seg_images)

In [ ]:
# Load meshes
meshes = []
meshes_path = data_path.replace("videos", "meshes")
for mesh_name in os.listdir(meshes_path):
    if not mesh_name.endswith(".obj"): continue
    mesh_path = os.path.join(meshes_path, mesh_name)
    renderer.add_mesh_from_file(mesh_path, force="mesh")
    meshes.append(mesh_name.replace(".obj", ""))
    print(meshes[-1])

In [ ]:
start_t = 11
seg_img = seg_images[start_t]

Image.fromarray(seg_img)

In [ ]:
num_objects = 10
indices, init_poses = [], []
obj_ids = jnp.unique(seg_img.reshape(-1, 3), axis=0)
obj_ids = sorted(
    obj_ids, key=lambda x: jnp.all(seg_img == x, axis=-1).sum(), reverse=True
)
for obj_id in obj_ids[: num_objects + 1]:
    if jnp.all(obj_id == 0):
        # Background
        continue

    obj_mask = jnp.all(seg_img == obj_id, axis=-1)
    masked_depth = coord_images[start_t].copy()
    masked_depth[~obj_mask] = 0

    object_points = coord_images[start_t][obj_mask]
    maxs = np.max(object_points, axis=0)
    mins = np.min(object_points, axis=0)
    dims = maxs - mins
    obj_center = (maxs + mins) / 2
    obj_transform = transform_from_pos(obj_center)

    best = utils.find_best_mesh(renderer, meshes, obj_transform, masked_depth)
    if best:
        indices.append(best[0])
        init_poses.append(best[1])

init_poses = jnp.array(init_poses)
rendered_image = renderer.render_multiobject(init_poses, indices)
get_depth_image(rendered_image[:, :, 2], max=5)

6. Define the liklihood methods and the proposal enumerations.

In [ ]:
# Liklihood parameters
r = radius = 0.1
outlier_prob = 0.005
outlier_volume = 0.5

# Enumeration parameters
n = 7  # number of enumerated proposals on each dimension (x, y, z).
d = 0.05  # the minimum and maximum position delta on each dimension (x, y, z).

# Enumerating proposals
translation_deltas_global = jax3dp3.make_translation_grid_enumeration(
    -d, -d, -d, d, d, d, n, n, n
)

reward_d = 0.3
translation_deltas_reward = jax3dp3.make_translation_grid_enumeration(
    -reward_d, -reward_d, -reward_d, reward_d, reward_d, reward_d, n, n, n
)
reward_deltas_mask = jnp.abs(translation_deltas_reward[:, -2, -1]) > 1e-6
translation_deltas_reward = translation_deltas_reward.at[reward_deltas_mask, -2, -1].set(0)

def prior(new_pose, prev_pose):
    weight = jax.scipy.stats.norm.pdf(new_pose[:3,3] - (prev_pose[:3,3] + jnp.array([0.0, 0.2, 0.0])), loc=0, scale=0.1)
    return weight.sum()
prior_parallel = jax.jit(jax.vmap(prior, in_axes=(0, None)))


def scorer(rendered_image, gt):
    weight = jax3dp3.likelihood.threedp3_likelihood(
        gt, rendered_image, r, outlier_prob, outlier_volume
    )
    return weight
scorer_parallel = jax.jit(jax.vmap(scorer, in_axes=(0, None)))

In [ ]:
# # jax.scipy.stats.norm.pdf(jnp.array([0, 0, 0]))
# old = [0, 0, 0]
# new = [-0.05, -0.05, 0]


# old = jnp.array(old)
# new = jnp.array(new)
# jnp.round(jax.scipy.stats.norm.pdf(new - (old + jnp.array([0.0, 0.2, 0.0])), loc=0, scale=0.1), 4)

In [ ]:
pose_estimates = init_poses.copy()
t = start_t
gt_image = jnp.array(coord_images[t])

translation_deltas = translation_deltas_global
translation_deltas_full = jnp.tile(
    jnp.eye(4)[None, :, :],
    (translation_deltas.shape[0], pose_estimates.shape[0], 1, 1),
)
translation_deltas_full = translation_deltas_full.at[:, 0, :, :].set(translation_deltas)
translation_proposals = jnp.einsum(
    "bij,abjk->abik", pose_estimates, translation_deltas_full
)
images = renderer.render_parallel(translation_proposals, 0)
weights_new = scorer_parallel(images, gt_image)
pose_estimates = translation_proposals[jnp.argmax(weights_new)]

In [ ]:
num_steps = num_frames - start_t
# num_steps = 50
iterations_per_step = 3
n_objects = pose_estimates.shape[0]
reward_idx = n_objects - 1

containment_relations = {}
objs_deltas = {}

inferred_poses = []
pose_estimates = init_poses.copy()
for t in tqdm(range(start_t, start_t + num_steps)):
    gt_image = jnp.array(coord_images[t])    
    for _ in range(iterations_per_step):
        for i in range(n_objects):
            if i in set(containment_relations.values()):
                continue
            translation_deltas = translation_deltas_global if i != reward_idx else translation_deltas_reward
                        
            # Check for occlusion
            if i == reward_idx:
                occluded = utils.check_occlusion(renderer, pose_estimates, indices, i)
                if occluded:
                    containing_obj = utils.check_containment(renderer, pose_estimates, indices, i)
                    if containing_obj is not None:
                        containment_relations[containing_obj] = i
                        continue
            
            translation_deltas_full = jnp.tile(
                jnp.eye(4)[None, :, :],
                (translation_deltas.shape[0], pose_estimates.shape[0], 1, 1),
            )
            translation_deltas_full = translation_deltas_full.at[:, i, :, :].set(translation_deltas)
            translation_proposals = jnp.einsum(
                "bij,abjk->abik", pose_estimates, translation_deltas_full
            )
            images = renderer.render_multiobject_parallel(translation_proposals.transpose((1,0,2,3)), indices)

            weights = scorer_parallel(images, gt_image) + prior_parallel(translation_proposals[:,i], pose_estimates[i])
            # weights = scorer_parallel(images, gt_image)
            best_weight_idx = jnp.argmax(weights)
            best_proposal = translation_proposals[best_weight_idx]
            
            objs_deltas[i] = best_proposal[i] - pose_estimates[i]
            pose_estimates = best_proposal
    
        for i, j in containment_relations.items():
            new_pose_estimate = pose_estimates[j] + objs_deltas[i]
            pose_estimates = pose_estimates.at[j].set(new_pose_estimate)
        
    inferred_poses.append(pose_estimates.copy())

In [ ]:
all_images = []
for t in tqdm(range(start_t, start_t + num_steps)):
    rgb_viz = Image.fromarray(rgb_images[t].astype(np.int8), mode="RGB")
    gt_depth_1 = get_depth_image(coord_images[t][:, :, 2], max=5.0)
    poses = inferred_poses[t - start_t]
    rendered_image = renderer.render_multiobject(poses, indices)
    rendered_image = get_depth_image(rendered_image[:, :, 2], max=5)

    apple_pose = poses[-1]
    rendered_apple = renderer.render_single_object(apple_pose, indices[-1])
    rendered_apple = [get_depth_image(rendered_apple[:, :, 2], max=5)]
    
    all_images.append(
        multi_panel(
            [rgb_viz, gt_depth_1, rendered_image, *rendered_apple],
            [
                f"\nRGB Image",
                f"   Frame: {t}\nActual Depth",
                "\nReconstructed Depth",
                *(["\nApple Only"] * len(rendered_apple)),
            ],
            middle_width=10,
            label_fontsize=20,
        )
    )
out_path = f"{scene}_out.gif"
make_gif_from_pil_images(all_images, out_path)
print("Saved output to:", out_path)